# TASK 1 : Install Dependencies & Run a SparkSession


In [1]:
#install pyspark
!pip install pyspark

     |████████████████████████████████| 204.2MB 60kB/s 
     |████████████████████████████████| 204kB 42.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=722c6e422fd0f86e25e5bd10cc8a757b26ebb0b87a9c363015aaf40958165b5a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
#create a sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("spark").getOrCreate()

# TASK 2 : Clone & Explore dataset

In [4]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [5]:
#check the presence of dataset
!ls admission_dataset

Admission_Predict_Ver1.1.csv


In [10]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv',header=True,inferSchema=True)

In [13]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [14]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [15]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [18]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [20]:
#drop the unnecessary column
df = df.drop('Serial No')

In [21]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [24]:
#check for null values
for i in df.columns:
  print(i+":",df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [29]:
# correlation analysis
for col in df.columns:
  print('Correlation to chance of admit for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation to chance of admit for GRE Score is 0.8103506354632598
Correlation to chance of admit for TOEFL Score is 0.7922276143050823
Correlation to chance of admit for University Rating is 0.6901323687886892
Correlation to chance of admit for SOP is 0.6841365241316723
Correlation to chance of admit for LOR is 0.6453645135280112
Correlation to chance of admit for CGPA is 0.882412574904574
Correlation to chance of admit for Research is 0.5458710294711379
Correlation to chance of admit for Chance of Admit is 1.0


In [33]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA'],outputCol='features')

In [36]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [37]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data =  output_data.select('features','Chance of Admit')

In [38]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [39]:
#split the dataset into training and testing set
train,test = final_data.randomSplit([0.7,0.3])

In [41]:
#build & train the model
models = LinearRegression(featuresCol='features',labelCol='Chance of Admit')
model = models.fit(train)

In [42]:
#get coefficients & intercept
print('coefficient:',model.coefficients)
print('intercept:',model.intercept)

coefficient: [0.0024717457868279496,0.002316569638468854,0.145645160175683]
intercept: -1.5575779235372786


In [44]:
#get summary of the model
summary = model.summary
summary

In [46]:
#print the rmse & r2 score
print("RMSE",summary.rootMeanSquaredError)
print('r2 score',summary.r2)

RMSE 0.06177322886106282
r2 score 0.8082263162559354


# TASK 6 : Evaluate & Save the Model

In [47]:
#transform on the test data
predictions = model.transform(test)

In [49]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,104.0,7.46]|           0.45| 0.4866644919541827|
|  [295.0,93.0,7.2]|           0.46| 0.4356732132194874|
| [295.0,96.0,7.34]|           0.47|0.46301324455948945|
| [295.0,99.0,7.57]|           0.37| 0.5034613403153034|
| [296.0,95.0,7.54]|           0.44| 0.4922974527429853|
| [296.0,99.0,8.03]|           0.61| 0.5729298597829453|
| [297.0,96.0,7.43]|           0.34|0.48106480054895706|
| [297.0,96.0,7.89]|           0.43| 0.5480615742297712|
| [297.0,100.0,7.9]|           0.52| 0.5587843043854035|
| [298.0,92.0,7.88]|           0.51| 0.5398105898609666|
|  [298.0,99.0,7.6]|           0.46| 0.5152459324810577|
|[298.0,105.0,8.54]|           0.69| 0.6660518008770124|
|[299.0,100.0,8.02]|           0.63| 0.5812052151801412|
| [300.0,99.0,8.01]|           0.58| 0.5799039397267436|
| [301.0,98.0,8.03]|           

In [51]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='Chance of Admit')
print('r2 on the test data',evaluator.evaluate(predictions))

r2 on the test data 0.0640774737642653


In [52]:
#save the model
model.save('model')

In [53]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('model')